# agenda
1. Knowledge graph
2. Neo4j graph DB
3. Cypher query language
4. Graphdb with langchain 

In [31]:
# knowledge Graph
# it is sematic network .it is network of realworld entities eg:= events,situations,concepts 
# knowledge graph can divide sentence into 3 main components. components are
# 1.nodes 
# 2.edges
# 3.labels

# Note: User query in RAG are 2 types 1.keyword search,2.Dense vector search .
# Eg neo4j create Node ->CREATE(entity:Label({"key":value}))

In [41]:
! pip install --upgrade langchain langchain_groq langchain_experimental neo4j

Defaulting to user installation because normal site-packages is not writeable


In [33]:
# Grapg DB configuration
NEO4J_URI="neo4j+s://4d0b2494.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="KjTtPM7uZP3pEiSe8KgxTguclmQfTmqvloUWz_qZ_wU"

In [34]:
import os

os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [35]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B25C8C9430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B25C8C9F70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [44]:
from langchain_core.documents import Document

text="""
Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
doc=[Document(page_content=text)]
doc

[Document(metadata={}, page_content='\nMarie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.\nShe was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.\nHer husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.\nShe was, in 1906, the first woman to become a professor at the University of Paris.\n')]

In [45]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

llm_tranformer=LLMGraphTransformer(llm=llm)

graph_doc=llm_tranformer.convert_to_graph_documents(doc)


In [46]:
graph_doc

[GraphDocument(nodes=[Node(id='Marie Curie', type='Person', properties={}), Node(id='Pierre Curie', type='Person', properties={})], relationships=[Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='Pierre Curie', type='Person', properties={}), type='SPOUSE', properties={})], source=Document(metadata={}, page_content='\nMarie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.\nShe was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.\nHer husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.\nShe was, in 1906, the first woman to become a professor at the University of Paris.\n'))]

In [47]:
graph_doc[0].nodes

[Node(id='Marie Curie', type='Person', properties={}),
 Node(id='Pierre Curie', type='Person', properties={})]

In [48]:
graph_doc[0].relationships

[Relationship(source=Node(id='Marie Curie', type='Person', properties={}), target=Node(id='Pierre Curie', type='Person', properties={}), type='SPOUSE', properties={})]

In [49]:
movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [50]:
graph.query(movie_query)

[#E9A2]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.124.169.171', 7687)) (ResolvedIPv4Address(('34.124.169.171', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.1254666657316583s (Unable to retrieve routing information)


[]

In [51]:
graph.refresh_schema()

In [52]:
print(graph.schema)

Node properties:
Person {born: INTEGER, name: STRING}
Movie {release_on: INTEGER, title: STRING, id: STRING, released: DATE, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [54]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001B25C1C77A0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B25C8C9430>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B25C8C9F70>, model_name=

In [57]:
res=chain.invoke({"query":"Who was the director of movie GoldenEye"})
res



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Campbell'}]

> Finished chain.


{'query': 'Who was the director of movie GoldenEye',
 'result': 'Martin Campbell  \n'}